In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/hull-tactical-market-prediction/train.csv
/kaggle/input/hull-tactical-market-prediction/test.csv
/kaggle/input/hull-tactical-market-prediction/kaggle_evaluation/default_inference_server.py
/kaggle/input/hull-tactical-market-prediction/kaggle_evaluation/default_gateway.py
/kaggle/input/hull-tactical-market-prediction/kaggle_evaluation/__init__.py
/kaggle/input/hull-tactical-market-prediction/kaggle_evaluation/core/templates.py
/kaggle/input/hull-tactical-market-prediction/kaggle_evaluation/core/base_gateway.py
/kaggle/input/hull-tactical-market-prediction/kaggle_evaluation/core/relay.py
/kaggle/input/hull-tactical-market-prediction/kaggle_evaluation/core/kaggle_evaluation.proto
/kaggle/input/hull-tactical-market-prediction/kaggle_evaluation/core/__init__.py
/kaggle/input/hull-tactical-market-prediction/kaggle_evaluation/core/generated/kaggle_evaluation_pb2.py
/kaggle/input/hull-tactical-market-prediction/kaggle_evaluation/core/generated/kaggle_evaluation_pb2_grpc.py
/kaggl

## Imports

In [2]:
import os
from pathlib import Path
import datetime
from dataclasses import dataclass, asdict
import polars as pl
import numpy as np
from sklearn.preprocessing import StandardScaler
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import kaggle_evaluation.default_inference_server

!pip install /kaggle/input/pandas-ta-man/pandas_ta-0.3.14b.tar.gz.tar
import pandas_ta as ta

Processing /kaggle/input/pandas-ta-man/pandas_ta-0.3.14b.tar.gz.tar
  Preparing metadata (setup.py) ... done
  Created wheel for pandas_ta: filename=pandas_ta-0.3.14b0-py3-none-any.whl size=218910 sha256=410cd2a0b11da87486431c7a2841640fe0b9b23367945f9e5dd71a27306008c1
  Stored in directory: /root/.cache/pip/wheels/cf/67/d5/b79954a9750c845c41651cab5827d7b0ed96d7253de184984b
Successfully built pandas_ta


## Project Directories

In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/hull-tactical-market-prediction/train.csv
/kaggle/input/hull-tactical-market-prediction/test.csv
/kaggle/input/hull-tactical-market-prediction/kaggle_evaluation/default_inference_server.py
/kaggle/input/hull-tactical-market-prediction/kaggle_evaluation/default_gateway.py
/kaggle/input/hull-tactical-market-prediction/kaggle_evaluation/__init__.py
/kaggle/input/hull-tactical-market-prediction/kaggle_evaluation/core/templates.py
/kaggle/input/hull-tactical-market-prediction/kaggle_evaluation/core/base_gateway.py
/kaggle/input/hull-tactical-market-prediction/kaggle_evaluation/core/relay.py
/kaggle/input/hull-tactical-market-prediction/kaggle_evaluation/core/kaggle_evaluation.proto
/kaggle/input/hull-tactical-market-prediction/kaggle_evaluation/core/__init__.py
/kaggle/input/hull-tactical-market-prediction/kaggle_evaluation/core/generated/kaggle_evaluation_pb2.py
/kaggle/input/hull-tactical-market-prediction/kaggle_evaluation/core/generated/kaggle_evaluation_pb2_grpc.py
/kaggl

## Configurations

In [4]:
# ============ PATHS ============
DATA_PATH: Path = Path('/kaggle/input/hull-tactical-market-prediction/')
#DATA_PATH: Path = hull_tactical_market_prediction_path

# ============ MODEL CONFIGS ============
#CV: int = 10                                    # Number of cross validation folds in the model fitting
#L1_RATIO: float = 0.5                           # ElasticNet mixing parameter
ALPHAS: np.ndarray = np.logspace(-4, 2, 100)    # Constant that multiplies the penalty terms
MAX_ITER: int = 1000000                         # The maximum number of iterations
N_FUTURE=1                                      # Number of days into future to predict stock price
TIME_STEPS = 5                                  # Look-back period for LSTM

## Seed Everything

In [5]:
import random

def seed_everything(seed):
    random.seed(seed)
    np.random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)

seed_everything(237)
seed=237

## Dataset Loading

In [6]:
def load_trainset() -> pl.DataFrame:
    """
    Loads and preprocesses the training dataset.

    Returns:
        pl.DataFrame: The preprocessed training DataFrame.
    """
    file_path = os.path.join(DATA_PATH, "train.csv")
    return (
        pl.read_csv(file_path)
        .rename({'market_forward_excess_returns':'target'})
        .with_columns(
            pl.exclude('date_id').cast(pl.Float64, strict=False)
        )
        #.head(-10)
    )

def load_testset() -> pl.DataFrame:
    """
    Loads and preprocesses the testing dataset.

    Returns:
        pl.DataFrame: The preprocessed testing DataFrame.
    """
    file_path = os.path.join(DATA_PATH, "test.csv")
    return (
        pl.read_csv(file_path)
        .rename({'lagged_forward_returns':'target'})
        .with_columns(
            pl.exclude('date_id').cast(pl.Float64, strict=False)
        )
    )

def join_train_test_dataframes(train: pl.DataFrame, test: pl.DataFrame) -> pl.DataFrame:
    """
    Joins two dataframes by common columns and concatenates them vertically.

    Args:
        train (pl.DataFrame): The training DataFrame.
        test (pl.DataFrame): The testing DataFrame.

    Returns:
        pl.DataFrame: A single DataFrame with vertically stacked data from common columns.
    """
    common_columns: list[str] = [col for col in train.columns if col in test.columns]

    return pl.concat([train.select(common_columns), test.select(common_columns)], how="vertical")



## Generate Train and Test data

In [7]:
train: pl.DataFrame = load_trainset()
test: pl.DataFrame = load_testset()
print(train.tail(3))
print(test.tail(3))

df_train=train.to_pandas()
print (df_train.shape)


shape: (3, 98)
┌─────────┬─────┬─────┬─────┬───┬───────────┬─────────────────┬────────────────┬───────────┐
│ date_id ┆ D1  ┆ D2  ┆ D3  ┆ … ┆ V9        ┆ forward_returns ┆ risk_free_rate ┆ target    │
│ ---     ┆ --- ┆ --- ┆ --- ┆   ┆ ---       ┆ ---             ┆ ---            ┆ ---       │
│ i64     ┆ f64 ┆ f64 ┆ f64 ┆   ┆ f64       ┆ f64             ┆ f64            ┆ f64       │
╞═════════╪═════╪═════╪═════╪═══╪═══════════╪═════════════════╪════════════════╪═══════════╡
│ 9045    ┆ 0.0 ┆ 0.0 ┆ 0.0 ┆ … ┆ -0.411743 ┆ 0.001852        ┆ 0.000147       ┆ 0.001395  │
│ 9046    ┆ 0.0 ┆ 0.0 ┆ 0.0 ┆ … ┆ -0.468078 ┆ 0.003463        ┆ 0.000146       ┆ 0.003006  │
│ 9047    ┆ 0.0 ┆ 0.0 ┆ 0.0 ┆ … ┆ -0.4878   ┆ 0.000117        ┆ 0.000144       ┆ -0.000339 │
└─────────┴─────┴─────┴─────┴───┴───────────┴─────────────────┴────────────────┴───────────┘
shape: (3, 99)
┌─────────┬─────┬─────┬─────┬───┬───────────┬──────────┬─────────────────────┬─────────────────────┐
│ date_id ┆ D1  ┆ D2  ┆ D3  ┆ … 

## Target for LightGBM

In [8]:
def create_LightGBM_target(df):
    """
    Creates a Sharpe-Optimized 'Oracle' target.
    0 = Cash (Bearish)
    1 = Market (Neutral / Low Conviction Bull)
    2 = Leverage (High Conviction Bull)
    """
    df = df.copy()

    # ---------------------------------------------------------
    # 1. Define Thresholds (The "Buffer Zone")
    # ---------------------------------------------------------
    # We calculate the daily volatility (standard deviation) to know what a "big move" is.
    # A common rule of thumb:
    # - If return > 0.5 * Daily_Vol -> Worth Leveraging (2)
    # - If return < 0.0             -> Cash (0)
    # - In between                  -> Just Hold (1)

    # Calculate rolling volatility (backward looking, then shifted back to align with current row)
    # We use a small window to adapt to current regimes
    daily_vol = df['forward_returns'].rolling(window=20).std().bfill()

    threshold = 0.5 * daily_vol  # You can tune this (e.g., 0.2 to 1.0)

    # ---------------------------------------------------------
    # 2. Assign Initial Targets
    # ---------------------------------------------------------
    # Default to 1 (Hold)
    targets = pd.Series(1, index=df.index)

    # Set 0 (Cash) if return is negative
    targets[df['forward_returns'] < -threshold] = 0

    # Set 2 (Lev) ONLY if return is significantly positive (beats the threshold)
    targets[df['forward_returns'] > threshold] = 2

    # ---------------------------------------------------------
    # 3. The "Black Swan" Dampener (Your logic, improved)
    # ---------------------------------------------------------
    # If volatility is extreme, we remove the option to use Leverage (2).
    # We force it down to 1 (Hold) or even 0 (Cash) if you want to be defensive.

    # Calculate Z-Score of Volatility
    rolling_vol_20 = df['forward_returns'].rolling(window=20).std()
    long_term_mean = rolling_vol_20.rolling(window=252).mean()
    long_term_std = rolling_vol_20.rolling(window=252).std()

    vol_z_score = (rolling_vol_20 - long_term_mean) / (long_term_std + 1e-6)

    # Alignment: We are creating a TARGET (Oracle), so we are allowed to peek at the
    # volatility of the *period we are predicting*.
    # So we shift the z-score backwards to match the target date.
    future_vol_z_score = vol_z_score.shift(-1).fillna(0)

    # Rule: If future volatility is > 2 Sigma, forbid Leverage.
    mask_extreme_vol = future_vol_z_score > 2.0

    # If it was 2 and vol is crazy, downgrade to 1
    targets = np.where((mask_extreme_vol) & (targets == 2), 1, targets)

    # ---------------------------------------------------------
    # 4. CRITICAL: Return Integers for Multiclass
    # ---------------------------------------------------------
    return targets.astype(int)

In [9]:
def create_LightGBM_target_inference(df): 
    """
    Inference version: Grades the PAST based on the CURRENT data.
    """
    df = df.copy()

    # 1. IDENTIFY THE TRUTH
    # In inference, 'lagged_forward_returns' at Index T IS the "Forward Return" of Index T-1.
    # We calculate metrics directly on this column.
    series_to_grade = df['lagged_forward_returns']

    # 2. DEFINE THRESHOLDS (Same logic as training)
    # Calculate volatility of the returns that JUST happened.
    daily_vol = series_to_grade.rolling(window=20).std().bfill()
    threshold = 0.5 * daily_vol 

    # 3. ASSIGN "RAW" TARGETS
    # Note: These targets sit at Index T, but they describe the ideal action for Index T-1.
    raw_targets = pd.Series(1, index=df.index) # Default Hold
    
    # If Today's return was bad, Yesterday should have been Cash
    raw_targets[series_to_grade < -threshold] = 0
    
    # If Today's return was great, Yesterday should have been Lev
    raw_targets[series_to_grade > threshold] = 2

    # 4. BLACK SWAN DAMPENER (Corrected for Inference)
    # We don't need shift(-1) here because 'series_to_grade' IS already the future relative to T-1.
    # We just check if the return that just occurred was wildly volatile relative to history.
    
    rolling_vol_20 = series_to_grade.rolling(window=20).std()
    long_term_mean = rolling_vol_20.rolling(window=252).mean()
    long_term_std = rolling_vol_20.rolling(window=252).std()
    
    # Z-Score of the volatility we just experienced
    current_vol_z_score = (rolling_vol_20 - long_term_mean) / (long_term_std + 1e-6)
    
    # If the move we just saw was a >2 Sigma event, we say "Yesterday shouldn't have leveraged"
    mask_extreme_vol = current_vol_z_score > 2.0
    
    raw_targets = np.where((mask_extreme_vol) & (raw_targets == 2), 1, raw_targets)

    # 5. THE FINAL SHIFT (Crucial Step)
    # We calculated the answer for T-1, but the answer is currently sitting in Row T.
    # We shift UP (-1) to put the answer into the correct row.
    
    # Row T-1 now contains the target derived from Row T's return.
    # astype('Int64') allows for NaN values, which is appropriate for the last row where the target is unknown
    pos_target_series = pd.Series(raw_targets).shift(-1).astype('Int64')
    
    # Note: The very last row (Today) will be NaN because we don't know Tomorrow's return yet.
    # This is correct. We only need pos_target for history.
    
    return pos_target_series

## Custom eval function (Competition metric)

In [10]:
class CompetitionMetric:
    def __init__(self, validation_df):
        # We store the raw returns and risk-free rate for the validation set
        self.returns = validation_df['forward_returns'].values
        self.risk_free = validation_df['risk_free_rate'].values
        self.market_returns = validation_df['forward_returns'].values

    def feval(self, preds, train_data):
        """
        Full Competition Metric (Sharpe + Vol Penalty + Return Penalty)
        """

        n_rows = len(self.returns)
        num_class = 3  # Hardcoded based on LightGBM objective='multiclass' and num_class=3

        # Reshape predictions to (n_rows, num_class) regardless of initial shape
        # This handles both (n_rows * num_class,) and (n_rows, num_class) inputs
        preds_prob = preds.reshape((n_rows, num_class))

        # Convert Probabilities to Weighted Position
        # 0*Prob(0) + 1*Prob(1) + 2*Prob(2)
        # Assuming class 0 is 0x leverage, class 1 is 1x leverage, class 2 is 2x leverage
        position = (preds_prob[:, 1] * 1.0) + (preds_prob[:, 2] * 2.0)

        # 2. Calculate Strategy Returns (Daily)
        # Formula: Risk_Free * (Uninvested Part) + Position * (Market Return)
        strategy_rets = self.risk_free * (1 - position) + position * self.returns

        # 3. Calculate Excess Returns
        strat_excess = strategy_rets - self.risk_free
        market_excess = self.market_returns - self.risk_free

        # 4. Calculate Annualized Stats (Approximate geometric mean for speed)
        # Note: In the official metric, they use geometric mean: (1+r).prod()**(1/N) - 1
        # For monitoring, arithmetic mean is usually 'close enough', but here is the exact math:

        strat_geom_mean = (1 + strat_excess).prod()**(1/len(strat_excess)) - 1
        market_geom_mean = (1 + market_excess).prod()**(1/len(market_excess)) - 1

        strat_std = np.std(strategy_rets) # Std deviation of raw returns usually
        market_std = np.std(self.market_returns)

        if strat_std == 0: strat_std = 1e-6
        if market_std == 0: market_std = 1e-6

        # Annualize
        trading_days = 252
        sharpe = (strat_geom_mean / strat_std) * np.sqrt(trading_days)

        strat_vol_annual = strat_std * np.sqrt(trading_days) * 100
        market_vol_annual = market_std * np.sqrt(trading_days) * 100

        # -----------------------------------------------------------
        # 5. The Penalties
        # -----------------------------------------------------------

        # A. Volatility Penalty
        # Punishment if Strategy Vol > 1.2x Market Vol
        vol_ratio = strat_vol_annual / market_vol_annual
        excess_vol = max(0, vol_ratio - 1.2)
        vol_penalty = 1 + excess_vol

        # B. Return Penalty (The missing piece)
        # Punishment if Strategy Return < Market Return
        # We assume the gap is calculated based on annual percentage differences
        return_gap = max(0, (market_geom_mean - strat_geom_mean) * 100 * trading_days)
        return_penalty = 1 + (return_gap**2) / 100

        # 6. Final Score
        final_score = sharpe / (vol_penalty * return_penalty)

        return 'comp_score', final_score, True

## Feature generation

In [11]:
def add_return_target_lags(df, df_LightGBM_target):
    df = df.copy()

    # 1. The Critical Lags (Target & Return)
    # We lag the 'forward_returns' to create 'past_returns'
    # Note: 'forward_returns' at T is the return for T+1.
    # So shift(1) gives us the return at T (Today's close).
    # shift(2) gives us T-1 (Yesterday).

    # Return Lags
    df['forward_returns_lagged_1'] = df['lagged_forward_returns'].shift(1) # Return realized today
    df['forward_returns_lagged_2'] = df['lagged_forward_returns'].shift(2) # Return realized yesterday
    df['forward_returns_lagged_3'] = df['lagged_forward_returns'].shift(3)
    df['forward_returns_lagged_5'] = df['lagged_forward_returns'].shift(5) # Weekly trend

    # Target Lags (What was the ideal position yesterday?)
    # Assuming 'target' is aligned to T
    df['pos_target_t-1'] = df_LightGBM_target.shift(1)
    df['pos_target_t-2'] = df_LightGBM_target.shift(2)
    df['pos_target_t-3'] = df_LightGBM_target.shift(3)

    # 2. Volatility (Derived)
    # 5-day rolling volatility of returns
    df['rolling_vol_5'] = df['forward_returns_lagged_1'].rolling(5).std()

    return df


In [12]:

def add_synthetic_price_features(df):
    # 1. Recover the "Past Returns"
    # forward_returns at index T represents the return for T+1.
    # So shift(1) gives us the return realized at T (Today).
    # We fill the first NaN with 0 to start the chain.
    realized_returns = df['lagged_forward_returns']

    # Fill NaN values in realized_returns before calculating cumulative product
    # A common approach is to fill with 0, implying no change for missing data.
    realized_returns_cleaned = realized_returns.fillna(0)

    # 2. Build Synthetic Price Index (Start at 100)
    # cumprod() compounds the returns to create a price curve
    synthetic_price = 100 * (1 + realized_returns_cleaned).cumprod()

    # Ensure synthetic_price is fully populated after cumprod
    synthetic_price = synthetic_price.ffill().bfill()

    # Check if there's enough data for technical indicators
    if len(synthetic_price) < 20: # Minimum length for BBands and RSI
        df['syn_rsi_14'] = np.nan
        df['syn_ppo_hist'] = np.nan
        df['syn_ppo_line'] = np.nan
        df['syn_bb_width'] = np.nan
        return df

    # 3. Calculate RSI (Scale Invariant - Safe)
    df['syn_rsi_14'] = ta.rsi(synthetic_price, length=14)

    # 4. Calculate PPO (Percentage Price Oscillator) instead of MACD
    # Why? Regular MACD depends on the absolute price (100 vs 4000).
    # PPO is the percentage version of MACD, so it works perfectly
    # on a synthetic price regardless of where it starts.
    ppo = ta.ppo(synthetic_price)
    df['syn_ppo_hist'] = ppo['PPOh_12_26_9'] # The Histogram
    df['syn_ppo_line'] = ppo['PPO_12_26_9']  # The Line

    # 5. (Optional) Bollinger Bands (Measure Volatility squeeze)
    bbands = ta.bbands(synthetic_price, length=20)

    # Safely access 'BBB_20_2.0'
    if 'BBB_20_2.0' in bbands.columns:
        df['syn_bb_width'] = bbands['BBB_20_2.0']
    else:
        df['syn_bb_width'] = np.nan
        print(f"Warning: 'BBB_20_2.0' not found in bbands output for {df.name if hasattr(df, 'name') else 'a series'}. Synthetic price length: {len(synthetic_price)}. This may be due to insufficient data or constant values.")

    return df




In [13]:
def add_indicator_lags(df):
    # List of new synthetic features you just created
    # (Adjust names if you named them differently)
    indicators = ['syn_rsi_14', 'syn_ppo_hist', 'syn_ppo_line',  'syn_bb_width']

    for col in indicators:
        if col in df.columns:
            # Only lag-1 is usually needed for "Slope" detection
            df[f'{col}_lag_1'] = df[col].shift(1)

            # Optional: lag-2 if you want to detect curvature (acceleration)
            # df[f'{col}_lag_2'] = df[col].shift(2)

    return df


In [14]:
def generate_features(df_original, isTraining):
    
    df = df_original.copy()
    
    if isTraining:
      df_LightGBM_target = pd.Series(create_LightGBM_target(df))
    else:
      df_LightGBM_target = pd.Series(create_LightGBM_target_inference(df))

    if isTraining:
      df['lagged_risk_free_rate'] = df['risk_free_rate'].shift()
      df['lagged_forward_returns'] = df['forward_returns'].shift()
      # The target, risk_free_rate, and forward_returns columns are raw inputs and will be dropped later
      # We keep them for now for creating lagged features or other computations.
    else: # Inference
      # 'is_scored' and 'lagged_market_forward_excess_returns' are specific to test data, not features
      df.drop(columns=['is_scored', 'lagged_market_forward_excess_returns'], inplace=True, errors='ignore')

    print(f"Shape before adding new features: {df.shape}")
    df_lags = add_return_target_lags(df, df_LightGBM_target)
    df_synt_lags = add_synthetic_price_features(df_lags).pipe(add_indicator_lags)

    # Define all columns that should NOT be part of the final feature set for the model
    columns_to_drop_from_X = [
        'date_id',
        'target',                 # Original target (from train) or test's 'target' (which is lagged_forward_returns)
        'risk_free_rate',         # Raw value, its lagged version is kept
        'forward_returns',        # Raw value, its lagged version is kept
        # 'is_scored' and 'lagged_market_forward_excess_returns' are dropped earlier for inference path
        # or don't exist in training path, handled by errors='ignore'
    ]

    # Drop these non-feature columns consistently for both training and inference
    df_final_features = df_synt_lags.drop(columns=columns_to_drop_from_X, errors='ignore')

    print(f"Shape after feature generation and dropping non-features: {df_final_features.shape}")
    return df_final_features, df_LightGBM_target

## Train LightGBM

In [15]:
import lightgbm as lgb
# ---------------------------------------------------------------------------------
# 2. The Training Loop
# ---------------------------------------------------------------------------------

def train_lightgbm_policy(X, y, feature_names, num_boost_round, best_params, weight_min):
    """
    X: Input Features (LSTM Hidden States + Lags + Macro)
    y: The Oracle Target (0 to 2)
    df_context: DataFrame containing raw returns for the metric calculation
    """

    models = []

    # -------------------------------------------------------
    # B. WEIGHTING: Prioritize Recent Data
    # -------------------------------------------------------
    # Create a linear decay weight.
    # Recent data (end of train) gets weight 1.0, oldest gets 0.5
    w_train = np.linspace(weight_min, 1.0, len(X))

    # C. Create Datasets (Pass weights here!)
    dtrain = lgb.Dataset(X, label=y,
                          feature_name=feature_names, weight=w_train)

    # E. Train

    model = lgb.train(
        best_params,
        dtrain,
        num_boost_round=num_boost_round,
    )

    models.append(model)

    return models


In [16]:
df_features_train, df_LightGBM_target = generate_features(df_train.copy(), isTraining=True)
X_final = df_features_train.iloc[:-1]
y_final = df_LightGBM_target.iloc[:-1]
display(X_final.shape)
display(y_final.shape)

Shape before adding new features: (9048, 100)
Shape after feature generation and dropping non-features: (9048, 112)


/usr/local/lib/python3.11/dist-packages/pandas/core/computation/expressions.py:73: RuntimeWarning: invalid value encountered in less
  return op(a, b)
/usr/local/lib/python3.11/dist-packages/pandas/core/computation/expressions.py:73: RuntimeWarning: invalid value encountered in greater
  return op(a, b)


(9047, 112)

(9047,)

#### Best params weighted - rolling windows

{'num_leaves': 137,
 'max_depth': 8,
 'min_data_in_leaf': 70,
 'learning_rate': 0.0322378071317859,
 'lambda_l1': 0.005456545606883056,
 'lambda_l2': 0.05908594087517651,
 'feature_fraction': 0.8393936179595116,
 'bagging_fraction': 0.9491692279948203,
 'bagging_freq': 4}

In [17]:
#best_params_weighted=study.best_params
best_params_weighted = {
 'num_leaves': 137,
 'max_depth': 8,
 'min_data_in_leaf': 70,
 'learning_rate': 0.0322378071317859,
 'lambda_l1': 0.005456545606883056,
 'lambda_l2': 0.05908594087517651,
 'feature_fraction': 0.8393936179595116,
 'bagging_fraction': 0.9491692279948203,
 'bagging_freq': 4,
 'objective': 'multiclass',
 'metric': 'multi_logloss',
 'num_class': 3,
 'verbosity': -1,
 'boosting_type': 'gbdt',
 'n_jobs': -1}


In [18]:
models_LightGBM_weighted = train_lightgbm_policy(X_final, y_final,
                                                 X_final.columns.tolist(),
                                                 num_boost_round = 90, best_params=best_params_weighted,weight_min=0.5 )


#### Best params unweighted, expanding windows , 3 folds ####


best_params_unweighted_expanding = {'num_leaves': 149,
                                    'max_depth': 7,
                                    'min_data_in_leaf': 54,
                                    'learning_rate': 0.0345,
                                    'lambda_l1': 0.8426,
                                    'lambda_l2': 0.0611,
                                    'feature_fraction': 0.9498,
                                    'bagging_fraction': 0.8619,
                                    'bagging_freq': 6}
best_params_unweighted_expanding['objective'] = 'multiclass',
best_params_unweighted_expanding['metric'] = 'multi_logloss',
best_params_unweighted_expanding['num_class']= 3,
best_params_unweighted_expanding['verbosity'] = -1
best_params_unweighted_expanding['boosting_type'] = 'gbdt'
best_params_unweighted_expanding['n_jobs'] = -1


--- Training Fold 1 ---
Training until validation scores don't improve for 100 rounds
[50]	valid_0's multi_logloss: 1.02854	valid_0's comp_score: 2.52809
[100]	valid_0's multi_logloss: 1.02798	valid_0's comp_score: 2.5133
[150]	valid_0's multi_logloss: 1.03308	valid_0's comp_score: 2.51957
Early stopping, best iteration is:
[71]	valid_0's multi_logloss: 1.02161	valid_0's comp_score: 2.53852
Evaluated only: multi_logloss
--- Training Fold 2 ---
Training until validation scores don't improve for 100 rounds
[50]	valid_0's multi_logloss: 1.05794	valid_0's comp_score: 0.215383
[100]	valid_0's multi_logloss: 1.05924	valid_0's comp_score: 0.165959
Early stopping, best iteration is:
[36]	valid_0's multi_logloss: 1.0539	valid_0's comp_score: 0.246042
Evaluated only: multi_logloss
--- Training Fold 3 ---
Training until validation scores don't improve for 100 rounds
[50]	valid_0's multi_logloss: 0.99313	valid_0's comp_score: 1.30788
[100]	valid_0's multi_logloss: 0.987647	valid_0's comp_score: 1.29678
[150]	valid_0's multi_logloss: 0.979411	valid_0's comp_score: 1.36566
[200]	valid_0's multi_logloss: 0.985436	valid_0's comp_score: 1.34281
Early stopping, best iteration is:
[122]	valid_0's multi_logloss: 0.978066	valid_0's comp_score: 1.35932
Evaluated only: multi_logloss


In [19]:
best_params_unweighted_expanding = {'num_leaves': 149,
                                    'max_depth': 7,
                                    'min_data_in_leaf': 54,
                                    'learning_rate': 0.0345,
                                    'lambda_l1': 0.8426,
                                    'lambda_l2': 0.0611,
                                    'feature_fraction': 0.9498,
                                    'bagging_fraction': 0.8619,
                                    'bagging_freq': 6}
best_params_unweighted_expanding['objective'] = 'multiclass',
best_params_unweighted_expanding['metric'] = 'multi_logloss',
best_params_unweighted_expanding['num_class']= 3,
best_params_unweighted_expanding['verbosity'] = -1
best_params_unweighted_expanding['boosting_type'] = 'gbdt'
best_params_unweighted_expanding['n_jobs'] = -1

In [20]:
models_LightGBM_unweighted_expanding  = train_lightgbm_policy(X_final, y_final,
                                                 X_final.columns.tolist(),
                                                 num_boost_round = 40, best_params=best_params_unweighted_expanding,weight_min=1)

## Inference

In [21]:
initial_history_df = df_train.copy()
initial_history_df['lagged_risk_free_rate'] = initial_history_df['risk_free_rate'].shift()
initial_history_df['lagged_forward_returns'] = initial_history_df['forward_returns'].shift()
initial_history_df.drop(columns=['target', 'risk_free_rate', 'forward_returns'], inplace=True)
initial_history_df=initial_history_df.tail(500)

## Predict

In [22]:
def predict(test: pl.DataFrame) -> float:
    # 1. Feature Engineering
    # Ensure this function generates the exact same columns as your training data!
    # (Drop the rename if it's not strictly needed for your create_example_dataset logic)
    # test = test.rename({'lagged_forward_returns':'target'})

    global initial_history_df
    df_test= test.to_pandas()
    # Corrected drop operation:
    #df_test = df_test.drop(columns=['lagged_market_forward_excess_returns'], errors='ignore')
    
    history_slice=initial_history_df[initial_history_df['date_id']<df_test['date_id'].iloc[0]]

    df_history_all=pd.concat([history_slice, df_test], axis=0, ignore_index=True)[-100:]
    X_test, _ = generate_features(df_history_all, isTraining=False) # Get only the features, ignore the target
    X_test=X_test.iloc[-1:]

    # 2. Get Probabilities
    # LightGBM .predict() returns shape (n_samples, 3)
    # We take [0] because we are predicting 1 row at a time in inference
    probs_weighted = models_LightGBM_weighted[0].predict(X_test)
    probs_unweighted_exp = models_LightGBM_unweighted_expanding[0].predict(X_test)
    probs_mean= (probs_weighted + probs_unweighted_exp) / 2

    # --- DEBUGGING BLOCK ---
    print("\n--- DEBUGGING STEP ---")
    print(f"Raw Input Timestamp: {df_test.index[0] if not df_test.index.empty else 'No Index'}")
    print(f"Features (First 3 cols): {X_test.iloc[0, :3].values}")
    print("Features shape ",X_test.shape)
    print(f"Model Output Probs: {probs_mean}")

    # Flatten probs_mean from (1, 3) to (3,) for a single prediction
    probs_mean = probs_mean[0]

    # probs is something like [0.10, 0.60, 0.30]
    # 3. Calculate Weighted Position
    # P(Hold)*1.0 + P(Buy)*2.0
    # (P(Sell) is multiplied by 0, so we ignore it)
    position_ensemble = (probs_mean[1] * 1.0) + (probs_mean[2] * 2.0)

    # 4. Clip for Safety (Strictly between 0 and 2)
    final_position = np.clip(position_ensemble, 0.0, 2.0)

    if len(initial_history_df) > 500:
        initial_history_df = initial_history_df.iloc[-500:]
        
    initial_history_df = pd.concat([initial_history_df, df_test], axis=0, ignore_index=True)

    return float(final_position)

## Launch Server

In [23]:
# 1. Initialize the Server
inference_server = kaggle_evaluation.default_inference_server.DefaultInferenceServer(predict)

# 2. The Logic Switch
if os.getenv('KAGGLE_IS_COMPETITION_RERUN'):
    # ==========================================
    # SCENARIO A: REAL SUBMISSION
    # ==========================================
    # Runs on the official hidden test set (which is already clean).
    print("🚀 Starting Official Submission Server...")
    inference_server.serve()
    
else:
    # ==========================================
    # SCENARIO B: SAVE & RUN ALL (The Fix)
    # ==========================================
    # We create a temporary "clean" folder to trick the Gateway into running
    # successfully so it generates the required 'submission.parquet' file.
    
    print("🛠️ Preparing clean test data for local simulation...")
    
    # A. Define paths
    local_data_path = '/kaggle/input/hull-tactical-market-prediction/'
    temp_dir = 'temp_data'
    os.makedirs(temp_dir, exist_ok=True)
    
    # B. Load the buggy test.csv
    # The error happens because 'is_scored' is Text instead of Number
    df_temp = pd.read_csv(os.path.join(local_data_path, 'test.csv'))
    
    # C. Fix the column (Force it to Integer)
    # We coerce errors to NaN, fill NaNs with 0, and cast to int
    if 'is_scored' in df_temp.columns:
        df_temp['is_scored'] = pd.to_numeric(df_temp['is_scored'], errors='coerce').fillna(0).astype(int)
    
    # D. Save the Clean Version to the temp folder
    df_temp.to_csv(os.path.join(temp_dir, 'test.csv'), index=False)
    
    # E. Copy the other necessary file (targets.csv) if it exists, or ignore
    # The gateway usually only strictly needs test.csv to start the loop
    
    print("✅ Clean data saved. Starting Local Gateway...")
    
    # F. Run Gateway pointing to the CLEAN folder
    # This generates 'submission.parquet' without crashing!
    try:
        inference_server.run_local_gateway((temp_dir,))
        print("✅ Success! submission.parquet generated.")
    except Exception as e:
        print(f"⚠️ Gateway Warning: {e}")
        # Even if it warns, check if submission.parquet exists
        if os.path.exists('submission.parquet'):
            print("   (File exists, so we are good to go!)")

🛠️ Preparing clean test data for local simulation...
✅ Clean data saved. Starting Local Gateway...
Shape before adding new features: (100, 97)
Shape after feature generation and dropping non-features: (100, 112)

--- DEBUGGING STEP ---
Raw Input Timestamp: 0
Features (First 3 cols): <FloatingArray>
[0.0, 0.0, 0.0]
Length: 3, dtype: Float64
Features shape  (1, 112)
Model Output Probs: [[0.24324007 0.43909402 0.31766591]]
Shape before adding new features: (100, 97)
Shape after feature generation and dropping non-features: (100, 112)

--- DEBUGGING STEP ---
Raw Input Timestamp: 0
Features (First 3 cols): <FloatingArray>
[0.0, 0.0, 0.0]
Length: 3, dtype: Float64
Features shape  (1, 112)
Model Output Probs: [[0.24648848 0.40601516 0.34749636]]
Shape before adding new features: (100, 97)
Shape after feature generation and dropping non-features: (100, 112)

--- DEBUGGING STEP ---
Raw Input Timestamp: 0
Features (First 3 cols): <FloatingArray>
[0.0, 0.0, 0.0]
Length: 3, dtype: Float64
Features

/usr/local/lib/python3.11/dist-packages/pandas/core/computation/expressions.py:73: RuntimeWarning: invalid value encountered in greater
  return op(a, b)
/usr/local/lib/python3.11/dist-packages/pandas/core/computation/expressions.py:73: RuntimeWarning: invalid value encountered in less
  return op(a, b)
/usr/local/lib/python3.11/dist-packages/pandas/core/computation/expressions.py:73: RuntimeWarning: invalid value encountered in greater
  return op(a, b)
/usr/local/lib/python3.11/dist-packages/pandas/core/computation/expressions.py:73: RuntimeWarning: invalid value encountered in greater
  return op(a, b)
/usr/local/lib/python3.11/dist-packages/pandas/core/computation/expressions.py:73: RuntimeWarning: invalid value encountered in less
  return op(a, b)
/usr/local/lib/python3.11/dist-packages/pandas/core/computation/expressions.py:73: RuntimeWarning: invalid value encountered in greater
  return op(a, b)
/usr/local/lib/python3.11/dist-packages/pandas/core/computation/expressions.py:73:


--- DEBUGGING STEP ---
Raw Input Timestamp: 0
Features (First 3 cols): <FloatingArray>
[0.0, 0.0, 0.0]
Length: 3, dtype: Float64
Features shape  (1, 112)
Model Output Probs: [[0.25452611 0.34625808 0.39921581]]
Shape before adding new features: (100, 97)
Shape after feature generation and dropping non-features: (100, 112)

--- DEBUGGING STEP ---
Raw Input Timestamp: 0
Features (First 3 cols): <FloatingArray>
[0.0, 0.0, 0.0]
Length: 3, dtype: Float64
Features shape  (1, 112)
Model Output Probs: [[0.19550976 0.47892211 0.32556813]]
Shape before adding new features: (100, 97)
Shape after feature generation and dropping non-features: (100, 112)

--- DEBUGGING STEP ---
Raw Input Timestamp: 0
Features (First 3 cols): <FloatingArray>
[0.0, 0.0, 0.0]
Length: 3, dtype: Float64
Features shape  (1, 112)
Model Output Probs: [[0.2036083  0.44712138 0.34927032]]
Shape before adding new features: (100, 97)
Shape after feature generation and dropping non-features: (100, 112)

--- DEBUGGING STEP ---
R

/usr/local/lib/python3.11/dist-packages/pandas/core/computation/expressions.py:73: RuntimeWarning: invalid value encountered in greater
  return op(a, b)
/usr/local/lib/python3.11/dist-packages/pandas/core/computation/expressions.py:73: RuntimeWarning: invalid value encountered in less
  return op(a, b)
/usr/local/lib/python3.11/dist-packages/pandas/core/computation/expressions.py:73: RuntimeWarning: invalid value encountered in greater
  return op(a, b)
/usr/local/lib/python3.11/dist-packages/pandas/core/computation/expressions.py:73: RuntimeWarning: invalid value encountered in greater
  return op(a, b)
/usr/local/lib/python3.11/dist-packages/pandas/core/computation/expressions.py:73: RuntimeWarning: invalid value encountered in less
  return op(a, b)
/usr/local/lib/python3.11/dist-packages/pandas/core/computation/expressions.py:73: RuntimeWarning: invalid value encountered in greater
  return op(a, b)
/usr/local/lib/python3.11/dist-packages/pandas/core/computation/expressions.py:73: